# IMPORT REQUIRED LIBRARIES

In [2]:
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
from torch.optim import Adam,lr_scheduler
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision import transforms
from PIL import Image
import os
import random
import time

# CUSTOMIZE THE DATASET INTO TRIPLETS

In [3]:
class Triplet:
    def __init__(self, train_folder):
        self.train_folder = train_folder
        self.labels = os.listdir(train_folder)
        self.label_to_path = {label: os.path.join(train_folder, label) for label in self.labels}
    
    def get_triplet(self):
        anchor_label = random.choice(self.labels)
        anchor_path = random.choice(os.listdir(self.label_to_path[anchor_label]))
        positive_label = anchor_label
        positive_path = random.choice(os.listdir(self.label_to_path[positive_label]))
        negative_label = random.choice([label for label in self.labels if label != anchor_label])
        negative_path = random.choice(os.listdir(self.label_to_path[negative_label]))
        
        anchor_image = os.path.join(self.label_to_path[anchor_label], anchor_path)
        positive_image = os.path.join(self.label_to_path[positive_label], positive_path)
        negative_image = os.path.join(self.label_to_path[negative_label], negative_path)
        
        return anchor_image, positive_image, negative_image

In [9]:
class TripletDataset(Dataset):
    def __init__(self, train_folder, transform=None):
        self.triplet_generator = Triplet(train_folder)
        self.transform = transform

    def __len__(self):
        return 5*12 # While Testing! Make it 50*12 for Training

    def __getitem__(self, index):
        anchor_image, positive_image, negative_image = self.triplet_generator.get_triplet()
        anchor = self._load_image(anchor_image)
        positive = self._load_image(positive_image)
        negative = self._load_image(negative_image)
        return anchor, positive, negative

    def _load_image(self, image_path):
        image = Image.open(image_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        return image

    def get_triplet_names(self, index):
        anchor_image, positive_image, negative_image = self.triplet_generator.get_triplet()
        return anchor_image, positive_image, negative_image

# DATALOADER

In [10]:
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert image to grayscale
    transforms.ToTensor(),
])

In [11]:
bs = 32
train_folder = "./Datasets/TAMIZH LETTERS/train"
dataset = TripletDataset(train_folder, transform=transform)
trainloader = DataLoader(dataset, batch_size=bs, shuffle=True)

In [12]:
len(trainloader)

2

# SIMILARITY CHECKING WITH A MODEL

In [13]:
import torch.nn as nn

class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 32, 5),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(32, 64, 5),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.fc = nn.Sequential(
            nn.Linear(64 * 53 * 53, 256),
            nn.PReLU(),
            nn.Linear(256, 256),
            nn.PReLU(),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def get_embedding(self, x):
        return self.forward(x)


In [14]:
emb = EmbeddingNet()

# TRIPLET WRAPPER

In [15]:
class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2=None, x3=None):
        if x2 is None and x3 is None:
            return self.embedding_net(x1)
        return self.embedding_net(x1),self.embedding_net(x2),self.embedding_net(x3)

    def get_embedding(self, x):
        return self.embedding_net(x)

# TRIPLET LOSS

In [16]:
class TripletLoss(nn.Module):
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin
    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean() if size_average else losses.sum()


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TripletNet(emb)
margin = 1
lr = 0.0001
n_epochs = 5
optimizer = Adam(model.parameters(), lr=lr)
loss_fn = TripletLoss(margin)

# TRAIN

In [18]:
def fit(model, num_epochs, train_loader,bs):
    for epoch in range(n_epochs):
        start = time.time()
        model.train()
        train_loss = 0.0
        for idx, batch in enumerate(train_loader):
            anchor, positive, negative = batch
            anchor = anchor.to(device)
            positive = positive.to(device)
            negative = negative.to(device)
            optimizer.zero_grad()
            anchor_embedding, positive_embedding, negative_embedding = model(anchor, positive, negative)
            anchor_embedding.requires_grad_(True)
            positive_embedding.requires_grad_(True)
            negative_embedding.requires_grad_(True)
            loss = loss_fn(anchor_embedding, positive_embedding, negative_embedding)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            print(f"({idx + 1}).  LOSS : {loss.item()}  SEEN : {bs * (idx + 1)}/{len(train_loader.dataset)}")

        print(f"Epoch {epoch + 1}/{n_epochs}, Train Loss: {train_loss / len(train_loader):.4f}, TIME: {time.time()-start}")

In [63]:
fit(model,n_epochs,trainloader,bs)

(1).  LOSS : 0.5223436951637268  SEEN : 32/600
(2).  LOSS : 0.27001625299453735  SEEN : 64/600
(3).  LOSS : 0.34986427426338196  SEEN : 96/600
(4).  LOSS : 0.19522054493427277  SEEN : 128/600
(5).  LOSS : 0.14254765212535858  SEEN : 160/600
(6).  LOSS : 0.41570398211479187  SEEN : 192/600
(7).  LOSS : 0.4501030445098877  SEEN : 224/600
(8).  LOSS : 0.40436965227127075  SEEN : 256/600
(9).  LOSS : 0.3919444680213928  SEEN : 288/600
(10).  LOSS : 0.3272921144962311  SEEN : 320/600
(11).  LOSS : 0.34925317764282227  SEEN : 352/600
(12).  LOSS : 0.27141156792640686  SEEN : 384/600
(13).  LOSS : 0.3114508092403412  SEEN : 416/600
(14).  LOSS : 0.24317187070846558  SEEN : 448/600
(15).  LOSS : 0.22839786112308502  SEEN : 480/600
(16).  LOSS : 0.376720666885376  SEEN : 512/600
(17).  LOSS : 0.16782569885253906  SEEN : 544/600
(18).  LOSS : 0.25150901079177856  SEEN : 576/600
(19).  LOSS : 0.24364502727985382  SEEN : 608/600
Epoch 1/5, Train Loss: 0.3112, TIME: 70.12755632400513
(1).  LOSS : 0

# EVALUATE

In [19]:
def evaluate_model(model, triplet_test_loader):
    model.eval()
    correct = 0
    total = 0
    start = time.time()
    with torch.no_grad():
        for (anchor, positive, negative) in triplet_test_loader:
            anchor_embedding, positive_embedding, negative_embedding = model(anchor, positive, negative)
            distance_positive = torch.norm(anchor_embedding - positive_embedding, dim=1)
            distance_negative = torch.norm(anchor_embedding - negative_embedding, dim=1)
            correct += torch.sum(distance_positive < distance_negative).item()
            total += anchor.size(0)
    accuracy = correct / total
    print(accuracy,time.time()-start)

In [20]:
test_folder = "./Datasets/TAMIZH LETTERS/test"
test_dataset = TripletDataset(test_folder, transform=transform)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [21]:
len(testloader)

2

# TESTING ACCURACY

In [28]:
evaluate_model(model,testloader)

0.85 2.3682732582092285


# SAVE

In [74]:
torch.save(model,"model/tripletTAMIZH.pt")

In [38]:
model_loaded = torch.load("model/tripletTAMIZH.pt")

# EVALUATING WITH FAISS

In [41]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

train_folder = "./Datasets/TAMIZH LETTERS/train/"
test_folder = "./Datasets/TAMIZH LETTERS/test/"

# Create the ImageFolder datasets for train and test folders
train_dataset = ImageFolder(train_folder, transform=transform)
test_dataset = ImageFolder(test_folder, transform=transform)

train_dataloader = DataLoader(train_dataset, shuffle=True)
test_dataloader = DataLoader(test_dataset, shuffle=False)


# FAISS LIBRARY

In [24]:
import faiss

In [25]:
import pickle

embs1 = None
labels1 = []

for idx, i in enumerate(train_dataloader):
    print(idx)
    I, L = i
    labels1.append(L.item())
    emb = model_loaded(I)  # Assuming `model_loaded(I)` returns a PyTorch tensor
    emb = emb.detach()
    if embs1 is None:
        embs1 = emb
    else:
        embs1 = torch.cat((embs1, emb), dim=0)

# Save the embedding vectors and labels
data = {
    'embs': embs1,
    'labels': labels1
}

with open('embedding_data.pkl', 'wb') as f:
    pickle.dump(data, f)


In [26]:
# Load the embedding vectors and labels
with open('embedding_data.pkl', 'rb') as f:
    data = pickle.load(f)

embs1 = data['embs']
labels1 = data['labels']


In [27]:
embs2 = None
labels2 = []
for i in test_dataloader:
    I, L = i
    labels2.append(L)
    emb = model_loaded(I)
    if embs2 is None:
        embs2 = emb
    else:
        embs2 = torch.cat((embs2, emb), dim=0)

# DIFFERENT INDICES

In [28]:
embs = embs1

In [29]:
index1 = faiss.IndexFlatL2(embs.shape[1])  # Assuming embs.shape[1] represents the dimensionality of the embeddings
index1.add(embs)

nlist = 100  # Number of cells/buckets
quantizer = faiss.IndexFlatL2(embs.shape[1])  # Quantizer index (same as IndexFlatL2)
index2 = faiss.IndexIVFFlat(quantizer, embs.shape[1], nlist)
index2.train(embs)
index2.add(embs)

index3 = faiss.IndexHNSWFlat(embs.shape[1], 32)  # M = 32 for the HNSW index
index3.add(embs)

nbits = 8  # Number of bits for the LSH hash
index4 = faiss.IndexLSH(embs.shape[1], nbits)
index4.add(embs)


# FAISS EVALUATION

In [30]:
def evaluatewithfaiss(embs,index):
    TOTAL = len(embs)
    CORRECT = 0
    start = time.time()
    for idx,emb in enumerate(embs):
        label = index.search(emb.detach().reshape(1,-1),1)[1][0][0]
        CORRECT += labels1[label]==labels2[idx]
    return f'{(CORRECT/TOTAL*100).item()}',f'TIME = {time.time()-start} SECONDS'
        

In [ ]:
print(f'IndexFlatL2 : {evaluatewithfaiss(embs2,index1)}')
print(f'IndexIVFFlat : {evaluatewithfaiss(embs2,index2)}')
print(f'IndexHNSWFlat : {evaluatewithfaiss(embs2,index3)}')
print(f'IndexLSH : {evaluatewithfaiss(embs2,index4)}')

# SIMPLE OPENCV APPLICATION

In [33]:
import torch
import torchvision.transforms as transforms
import cv2
from PIL import Image
import numpy as np
imgs = [
    "./Images/12.jpg", "./Images/1.jpg",
    "./Images/3.jpg", "./Images/2.jpg",
    "./Images/4.jpg", "./Images/5.jpg",
    "./Images/7.jpg","./Images/6.jpg",
    "./Images/8.jpg","./Images/9.jpg",
    "./Images/11.jpg","./Images/10.jpg",
]
x = [np.array(Image.open(img)) for img in imgs]  # Open image without conversion
y = [Image.open(img) for img in imgs]  # Open image without conversion

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [34]:
labelsd = {'0':"அ",
           '1':'ஆ',
           '2':'இ',
           '3':'ஈ',
          '4':'உ',
           '5':'ஊ',
           '6':'எ',
           '7':'ஏ',
           '8':'ஐ',
           '9':'ஒ',
           '10':'ஓ',
           '11':'ஔ'
          }

In [37]:
th = 0
while True:
    frame = cv2.resize(x[th], (900, 900))
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR for OpenCV display
    rect_height = 100
    cv2.rectangle(frame, (0, 0), (frame.shape[1], rect_height), (0, 0, 0), -1)
    image_tensor = transform(y[th])
    image_tensor = image_tensor.unsqueeze(0)
    emb = model_loaded(image_tensor)
    label = index2.search(emb.detach().reshape(1,-1),1)[1][0][0]
    value = labelsd[str(labels1[label])]
    text = f"TYPE : {labels1[label]} {value}"
    text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 2, 5)
    text_x = 10
    text_y = int(rect_height / 2) + int(text_size[1] / 2)
    cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 5)
    
    cv2.imshow('TAMIZH LETTERS', frame)
    k = cv2.waitKey()
    if k == 27:
        break
    elif k == 110 and th < 15:
        if th + 1 == len(imgs):
            th = 0
        else:
            th += 1
            
    elif k == 112 and th > 0:
        if th - 1 != -1:
            th -= 1

cv2.destroyAllWindows()

